# Noise correlation matrix

Hijacking of `noise_correlation` within class `Model` for computing the algebraic structure of matrix B for an arbitrary model.

In [2]:
%matplotlib inline
import numpy as np
import pyross
import copy
import matplotlib.pyplot as plt

### First define model: SIR with asymptomatic

In [129]:
SIR_model_spec = {
    "classes" : ["S", "Ia", "Is", "R"],

    "S" : {
        "infection" : [ ["Ia", "S", "-beta*alpha"],
                        ["Is", "S", "-beta*alpha"],
                        ["Ia", "S", "-beta*(1-alpha)"],
                        ["Is", "S", "-beta*(1-alpha)"] ]
    },
    
    "Ia" : {
        "linear"    : [["Ia", "-gIa"]]  ,
        "infection" : [ ["Ia", "S", "beta*alpha"],
                        ["Is", "S", "beta*alpha"] ]
    },
    
    "Is" : {
        "linear"    : [["Is", "-gIs"]]  ,
        "infection" : [ ["Ia", "S", "beta*(1-alpha)"],
                        ["Is", "S", "beta*(1-alpha)"] ]
    },
    
    "R" : {
        "linear"    : [ ["Ia", "gIa"],
                        ["Is", "gIs"] ]     
    }
}

def parameter_mapping(input_parameters, t):
    output_parameters = {
        'beta' : input_parameters['beta'],
        'gIa' : input_parameters['gIa'],
        'gIs' : input_parameters['gIs'],
        'alpha' : input_parameters['alpha'],
        'beta*alpha' : input_parameters['beta']*input_parameters['alpha'],
        'beta*(1-alpha)' : input_parameters['beta']*(1-input_parameters['alpha']),
    }        
    return output_parameters

SIR_parameters = {
    'beta' : 0.5,
    'gIa' : 0.5,
    'gIs' : 0.1,
    'alpha' : 0.4
}

M = 2
Ni = 1000*np.ones(M)
N = np.sum(Ni) 

E0 = np.array([10,0])
S0 = Ni-E0
I0 = np.array([0,0])
R0 = np.array([0,0])

x0 = {
    'S' : S0,
    'E' : E0,
    'I' : I0, 
    'R' : R0 
}

def contactMatrix(t):
    return np.identity(M)


In [130]:
# SIR model
model = pyross.stochastic.Model(SIR_model_spec, SIR_parameters, M, Ni,time_dep_param_mapping=parameter_mapping)
estimator = pyross.inference.Model(SIR_model_spec,SIR_parameters, M, Ni, time_dep_param_mapping=parameter_mapping)


### Compute noise correlation matrix

In [131]:
self = estimator

M = self.M
nClass = self.nClass
B = self.B
CM = self.CM
#parameters = self.model_parameters
parameters = self.param_keys
constant_terms = self.constant_terms
linear_terms = self.linear_terms
infection_terms = self.infection_terms
finres_terms = self.finres_terms
resource_list = self.resource_list
finres_pop = self.finres_pop
Omega = self.Omega
fi = self.fi
num_of_infection_terms = infection_terms.shape[0]


l = np.empty((num_of_infection_terms, self.M), dtype=object)
l[:] = ''
x = [var for var in self.model_spec['classes'] for i in range(M)]

for i in range(num_of_infection_terms):
    infective_index = infection_terms[i, 1]
    for m in range(M):
        for n in range(M):
            index = n + M*infective_index
            if fi[n]>0:
#                l[i, m] += CM[m,n]*x[index]/fi[n]
                l[i, m] = np.char.add(l[i, m],' + CM[' + str(m) + ',' + str(n) + ']*' + x[index] + '[' + str(n) + ']' + '/fi[' + str(n) + ']')

B_new = np.empty(B.shape, dtype=object)
B_new[:] = ''

    
if self.constant_terms.size > 0:
    for i in range(constant_terms.shape[0]):
        rate_index = constant_terms[i, 0]
        class_index = constant_terms[i, 1]
#        overdispersion_index = constant_terms[i, 3]
#        rate = parameters[rate_index]
#        rate = [self.param_keys[rate_index] for i in range(M)]
#        rate = self.param_keys[rate_index]
        rate = self.model_param_keys[rate_index]
#        if overdispersion_index == -1:
#            overdispersion = np.ones(M)
#        else:
#            overdispersion = parameters[overdispersion_index]
        for m in range(M):
#            B[class_index, m, class_index, m] += rate[m]*overdispersion[m]/Omega
#            B[nClass-1, m, nClass-1, m] += rate[m]*overdispersion[m]/Omega
            B_new[class_index, m, class_index, m] = np.char.add(B_new[class_index, m, class_index, m],
#                                                    ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate[m] + '*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']/Omega')
                                                    ' + ' + rate + '[' + str(m) + ']' + '/Omega')
            B_new[nClass-1, m, nClass-1, m] = np.char.add(B_new[nClass-1, m, nClass-1, m],
#                                                    ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate[m] + '*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']/Omega')
                                                    ' + ' + rate + '[' + str(m) + ']' + '/Omega')

for i in range(infection_terms.shape[0]):
    rate_index = infection_terms[i, 0]
    infective_index = infection_terms[i, 1]
    susceptible_index = infection_terms[i, 2]
    product_index = infection_terms[i, 3]
#    overdispersion_index = infection_terms[i, 4]
#    rate = parameters[rate_index]
#    rate = [self.param_keys[rate_index] for i in range(M)]
    rate = self.model_param_keys[rate_index]
#    s = x[susceptible_index*M:(susceptible_index+1)*M]
    s = list(self.class_index_dict.keys())[susceptible_index]
#    if overdispersion_index == -1:
#        overdispersion = np.ones(M)
#    else:
#        overdispersion = parameters[overdispersion_index]
    for m in range(M):
#        B[susceptible_index, m, susceptible_index, m] += rate[m]*overdispersion[m]*l[i, m]*s[m]
        B_new[susceptible_index, m, susceptible_index, m] = np.char.add(B_new[susceptible_index, m, susceptible_index, m],
#            ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' + ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')
        if product_index>-1:
#            B[susceptible_index, m, product_index, m] -=  rate[m]*overdispersion[m]*l[i, m]*s[m]
#            B[product_index, m, product_index, m] += rate[m]*overdispersion[m]*l[i, m]*s[m]
#            B[product_index, m, susceptible_index, m] -= rate[m]*overdispersion[m]*l[i, m]*s[m]
            B_new[susceptible_index, m, product_index, m] = np.char.add(B_new[susceptible_index, m, product_index, m],
#            ' - rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' - ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')
            B_new[product_index, m, product_index, m] = np.char.add(B_new[product_index, m, product_index, m],
#            ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' + ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')
            B_new[product_index, m, susceptible_index, m] = np.char.add(B_new[product_index, m, susceptible_index, m],
#            ' - rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' - ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')


for i in range(linear_terms.shape[0]):
    product_index = linear_terms[i, 2]
    reagent_index = linear_terms[i, 1]
#    overdispersion_index = linear_terms[i, 3]
#    reagent = x[reagent_index*M:(reagent_index+1)*M]
    reagent = list(self.class_index_dict.keys())[reagent_index]
    rate_index = linear_terms[i, 0]
#    rate = parameters[rate_index]
#    rate = [self.param_keys[rate_index] for i in range(M)]
#    rate = self.param_keys[rate_index]
    rate = self.model_param_keys[rate_index]
#    if overdispersion_index == -1:
#        overdispersion = np.ones(M)
#    else:
#        overdispersion = parameters[overdispersion_index]
    for m in range(M): # only fill in the upper triangular form
#        B[reagent_index, m, reagent_index, m] += rate[m]*overdispersion[m]*reagent[m]
        B_new[reagent_index, m, reagent_index, m] = np.char.add(B_new[reagent_index, m, reagent_index, m],
#                                     ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' + ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')
        if product_index>-1:
#            B[product_index, m, product_index, m] += rate[m]*overdispersion[m]*reagent[m]
#            B[reagent_index, m, product_index, m] += -rate[m]*overdispersion[m]*reagent[m]
#            B[product_index, m, reagent_index, m] += -rate[m]*overdispersion[m]*reagent[m]
            B_new[product_index, m, product_index, m] = np.char.add(B_new[product_index, m, product_index, m],
#                                     ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' + ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')
            B_new[reagent_index, m, product_index, m] = np.char.add(B_new[reagent_index, m, product_index, m],
#                                     ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' - ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')
            B_new[product_index, m, reagent_index, m] = np.char.add(B_new[product_index, m, reagent_index, m],
#                                     ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' - ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')


if finres_terms.size > 0:
    for i in range(finres_terms.shape[0]):
        resource_index = finres_terms[i, 0]
        rate_index = resource_list[resource_index][0]
        priority_index = finres_terms[i, 1]
        probability_index = finres_terms[i, 2]
        class_index = finres_terms[i, 3]
        reagent_index = finres_terms[i, 4]
        product_index = finres_terms[i, 5]
        overdispersion_index = finres_terms[i, 6]
        if overdispersion_index == -1:
            overdispersion = np.ones(M)
        else:
            overdispersion = parameters[overdispersion_index]
        for m in range(M):
            if np.size(finres_pop[resource_index]) == 1:
                frp = finres_pop[resource_index]
            else:
                frp = finres_pop[resource_index][m]
            term = parameters[rate_index, m] * parameters[priority_index, m] \
                   * parameters[probability_index, m] * overdispersion[m] * x[class_index*M+m] / (frp * self.Omega)
            if reagent_index>-1:
                B[reagent_index, m, reagent_index, m] += term
                if product_index>-1:
                    B[reagent_index, m, product_index, m] -= term
                    B[product_index, m, reagent_index, m] -= term
            if product_index>-1:
                B[product_index, m, product_index, m] += term



### Get / confirm results

In [132]:
m = 0
print(B_new[0,m,0,m])
print(B_new[0,m,1,m])
print(B_new[1,m,1,m])
print(B_new[0,m,2,m])
print(B_new[2,m,2,m])

 + beta*alpha[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] + beta*alpha[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0] + beta*(1-alpha)[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] + beta*(1-alpha)[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0]
 - beta*alpha[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] - beta*alpha[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0]
 + beta*alpha[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] + beta*alpha[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0] + gIa[0]*Ia[0]
 - beta*(1-alpha)[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] - beta*(1-alpha)[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0]
 + beta*(1-alpha)[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] + beta*(1-alpha)[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0] + gIs[0]*Is[0]


Confirm: SIR noise correlation matrix from `inference.pyx`:

    for m in range(M):
        for n in range(M):
            l[m] += beta[m]*CM[m,n]*(Ia[n]+fsa[n]*Is[n])/fi[n]

    for m in range(M): # only fill in the upper triangular form
        B[0, m, 0, m] = l[m]*s[m]
        B[0, m, 1, m] =  - alpha[m]*l[m]*s[m]
        B[1, m, 1, m] = alpha[m]*l[m]*s[m] + gIa[m]*Ia[m]
        B[0, m, 2, m] = - balpha[m]*l[m]*s[m]
        B[2, m, 2, m] = balpha[m]*l[m]*s[m] + gIs[m]*Is[m]


### Define model: SI2R with asymptomatic

In [139]:
SI2R_model_spec = {
    "classes" : ["S", "Ip", "Ia", "Is", "R"],

    "S" : {
        "infection" : [ ["Ip", "S", "-beta"],
                        ["Ia", "S", "-beta"],
                        ["Is", "S", "-beta"]]
    },
    
    "Ip" : {
        "linear"    : [["Ip", "-gIp*alpha"],
                        ["Ip", "-gIp*(1-alpha)"]]  ,
        "infection" : [ ["Ip", "S", "beta"],
                        ["Ia", "S", "beta"],
                        ["Is", "S", "beta"] ]
    },

    "Ia" : {
        "linear"    : [["Ip", "gIp*alpha"],
                      ["Ia", "-gIa"]]
    },
    
    "Is" : {
        "linear"    : [["Ip", "gIp*(1-alpha)"],
                      ["Is", "-gIs"]]
    },
    
    "R" : {
        "linear"    : [ ["Ia", "gIa"],
                        ["Is", "gIs"] ]     
    }
}

def parameter_mapping(input_parameters, t):
    output_parameters = {
        'beta' : input_parameters['beta'],
        'gIa' : input_parameters['gIa'],
        'gIs' : input_parameters['gIs'],
        'alpha' : input_parameters['alpha'],
        'gIp*alpha' : input_parameters['gIp']*input_parameters['alpha'],
        'gIp*(1-alpha)' : input_parameters['gIp']*(1-input_parameters['alpha']),
    }        
    return output_parameters

SI2R_parameters = {
    'beta' : 0.5,
    'gIp' : 0.5,
    'gIa' : 0.5,
    'gIs' : 0.1,
    'alpha' : 0.4
}

M = 2
Ni = 1000*np.ones(M)
N = np.sum(Ni) 

Ip0 = np.array([0,0])     # 
Ia0 = np.array([0,0])     # the SIR model has only one kind of infective 
Is0 = np.array([1,0])     # we take these to be symptomatic 
R0  = np.array([0,0])     # and assume there are no removed individuals initially 
S0  = N-(Ia0+Ip0+Is0+R0)    # so that the initial susceptibles are obtained from S + Ia + Is + R = N

x0 = {
    'S' : S0,
    'Ip' : Ip0,
    'Ia' : Is0, 
    'Is' : Is0, 
    'R' : R0 
}

def contactMatrix(t):
    return np.identity(M)


In [140]:
# SI2R model
model = pyross.stochastic.Model(SI2R_model_spec, SI2R_parameters, M, Ni,time_dep_param_mapping=parameter_mapping)
estimator = pyross.inference.Model(SI2R_model_spec,SI2R_parameters, M, Ni, time_dep_param_mapping=parameter_mapping)


### Compute noise correlation matrix

In [141]:
self = estimator

M = self.M
nClass = self.nClass
B = self.B
CM = self.CM
#parameters = self.model_parameters
parameters = self.param_keys
constant_terms = self.constant_terms
linear_terms = self.linear_terms
infection_terms = self.infection_terms
finres_terms = self.finres_terms
resource_list = self.resource_list
finres_pop = self.finres_pop
Omega = self.Omega
fi = self.fi
num_of_infection_terms = infection_terms.shape[0]


l = np.empty((num_of_infection_terms, self.M), dtype=object)
l[:] = ''
x = [var for var in self.model_spec['classes'] for i in range(M)]

for i in range(num_of_infection_terms):
    infective_index = infection_terms[i, 1]
    for m in range(M):
        for n in range(M):
            index = n + M*infective_index
            if fi[n]>0:
#                l[i, m] += CM[m,n]*x[index]/fi[n]
                l[i, m] = np.char.add(l[i, m],' + CM[' + str(m) + ',' + str(n) + ']*' + x[index] + '[' + str(n) + ']' + '/fi[' + str(n) + ']')

B_new = np.empty(B.shape, dtype=object)
B_new[:] = ''

    
if self.constant_terms.size > 0:
    for i in range(constant_terms.shape[0]):
        rate_index = constant_terms[i, 0]
        class_index = constant_terms[i, 1]
#        overdispersion_index = constant_terms[i, 3]
#        rate = parameters[rate_index]
#        rate = [self.param_keys[rate_index] for i in range(M)]
#        rate = self.param_keys[rate_index]
        rate = self.model_param_keys[rate_index]
#        if overdispersion_index == -1:
#            overdispersion = np.ones(M)
#        else:
#            overdispersion = parameters[overdispersion_index]
        for m in range(M):
#            B[class_index, m, class_index, m] += rate[m]*overdispersion[m]/Omega
#            B[nClass-1, m, nClass-1, m] += rate[m]*overdispersion[m]/Omega
            B_new[class_index, m, class_index, m] = np.char.add(B_new[class_index, m, class_index, m],
#                                                    ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate[m] + '*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']/Omega')
                                                    ' + ' + rate + '[' + str(m) + ']' + '/Omega')
            B_new[nClass-1, m, nClass-1, m] = np.char.add(B_new[nClass-1, m, nClass-1, m],
#                                                    ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate[m] + '*overdispersion[' + str(m) + ']/Omega')
#                                                    ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']/Omega')
                                                    ' + ' + rate + '[' + str(m) + ']' + '/Omega')

for i in range(infection_terms.shape[0]):
    rate_index = infection_terms[i, 0]
    infective_index = infection_terms[i, 1]
    susceptible_index = infection_terms[i, 2]
    product_index = infection_terms[i, 3]
#    overdispersion_index = infection_terms[i, 4]
#    rate = parameters[rate_index]
#    rate = [self.param_keys[rate_index] for i in range(M)]
    rate = self.model_param_keys[rate_index]
#    s = x[susceptible_index*M:(susceptible_index+1)*M]
    s = list(self.class_index_dict.keys())[susceptible_index]
#    if overdispersion_index == -1:
#        overdispersion = np.ones(M)
#    else:
#        overdispersion = parameters[overdispersion_index]
    for m in range(M):
#        B[susceptible_index, m, susceptible_index, m] += rate[m]*overdispersion[m]*l[i, m]*s[m]
        B_new[susceptible_index, m, susceptible_index, m] = np.char.add(B_new[susceptible_index, m, susceptible_index, m],
#            ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' + ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')
        if product_index>-1:
#            B[susceptible_index, m, product_index, m] -=  rate[m]*overdispersion[m]*l[i, m]*s[m]
#            B[product_index, m, product_index, m] += rate[m]*overdispersion[m]*l[i, m]*s[m]
#            B[product_index, m, susceptible_index, m] -= rate[m]*overdispersion[m]*l[i, m]*s[m]
            B_new[susceptible_index, m, product_index, m] = np.char.add(B_new[susceptible_index, m, product_index, m],
#            ' - rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' - ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')
            B_new[product_index, m, product_index, m] = np.char.add(B_new[product_index, m, product_index, m],
#            ' + rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' + ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' + ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')
            B_new[product_index, m, susceptible_index, m] = np.char.add(B_new[product_index, m, susceptible_index, m],
#            ' - rate[' + str(m) + ']*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*s[' + str(m) + ']')
#            ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
#            ' - ' + rate + '[' + str(m) + ']' + '*l[' + str(i) + ',' + str(m) + ']*' + s + '[' + str(m) + ']')
            ' - ' + rate + '[' + str(m) + ']' + '*(' + str(l[i,m]) + ')*' + s + '[' + str(m) + ']')


for i in range(linear_terms.shape[0]):
    product_index = linear_terms[i, 2]
    reagent_index = linear_terms[i, 1]
#    overdispersion_index = linear_terms[i, 3]
#    reagent = x[reagent_index*M:(reagent_index+1)*M]
    reagent = list(self.class_index_dict.keys())[reagent_index]
    rate_index = linear_terms[i, 0]
#    rate = parameters[rate_index]
#    rate = [self.param_keys[rate_index] for i in range(M)]
#    rate = self.param_keys[rate_index]
    rate = self.model_param_keys[rate_index]
#    if overdispersion_index == -1:
#        overdispersion = np.ones(M)
#    else:
#        overdispersion = parameters[overdispersion_index]
    for m in range(M): # only fill in the upper triangular form
#        B[reagent_index, m, reagent_index, m] += rate[m]*overdispersion[m]*reagent[m]
        B_new[reagent_index, m, reagent_index, m] = np.char.add(B_new[reagent_index, m, reagent_index, m],
#                                     ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' + ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')
        if product_index>-1:
#            B[product_index, m, product_index, m] += rate[m]*overdispersion[m]*reagent[m]
#            B[reagent_index, m, product_index, m] += -rate[m]*overdispersion[m]*reagent[m]
#            B[product_index, m, reagent_index, m] += -rate[m]*overdispersion[m]*reagent[m]
            B_new[product_index, m, product_index, m] = np.char.add(B_new[product_index, m, product_index, m],
#                                     ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' + ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' + ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' + ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')
            B_new[reagent_index, m, product_index, m] = np.char.add(B_new[reagent_index, m, product_index, m],
#                                     ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' - ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')
            B_new[product_index, m, reagent_index, m] = np.char.add(B_new[product_index, m, reagent_index, m],
#                                     ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*reagent[' + str(m) + ']')
#                             ' - ' + rate[m] + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
#                             ' - ' + rate + '[' + str(m) + ']' + '*overdispersion[' + str(m) + ']*' + reagent + '[' + str(m) + ']')
                             ' - ' + rate + '[' + str(m) + ']' + '*' + reagent + '[' + str(m) + ']')


if finres_terms.size > 0:
    for i in range(finres_terms.shape[0]):
        resource_index = finres_terms[i, 0]
        rate_index = resource_list[resource_index][0]
        priority_index = finres_terms[i, 1]
        probability_index = finres_terms[i, 2]
        class_index = finres_terms[i, 3]
        reagent_index = finres_terms[i, 4]
        product_index = finres_terms[i, 5]
        overdispersion_index = finres_terms[i, 6]
        if overdispersion_index == -1:
            overdispersion = np.ones(M)
        else:
            overdispersion = parameters[overdispersion_index]
        for m in range(M):
            if np.size(finres_pop[resource_index]) == 1:
                frp = finres_pop[resource_index]
            else:
                frp = finres_pop[resource_index][m]
            term = parameters[rate_index, m] * parameters[priority_index, m] \
                   * parameters[probability_index, m] * overdispersion[m] * x[class_index*M+m] / (frp * self.Omega)
            if reagent_index>-1:
                B[reagent_index, m, reagent_index, m] += term
                if product_index>-1:
                    B[reagent_index, m, product_index, m] -= term
                    B[product_index, m, reagent_index, m] -= term
            if product_index>-1:
                B[product_index, m, product_index, m] += term



### Get / confirm results

In [143]:
m = 0
print(B_new[0,m,0,m])
print(B_new[0,m,1,m])
print(B_new[0,m,2,m])
print(B_new[0,m,3,m])
print(B_new[1,m,1,m])
print(B_new[1,m,2,m])
print(B_new[1,m,3,m])
print(B_new[2,m,2,m])
print(B_new[2,m,3,m])
print(B_new[3,m,3,m])


 + beta[0]*( + CM[0,0]*Ip[0]/fi[0] + CM[0,1]*Ip[1]/fi[1])*S[0] + beta[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] + beta[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0]
 - beta[0]*( + CM[0,0]*Ip[0]/fi[0] + CM[0,1]*Ip[1]/fi[1])*S[0] - beta[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] - beta[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0]


 + beta[0]*( + CM[0,0]*Ip[0]/fi[0] + CM[0,1]*Ip[1]/fi[1])*S[0] + beta[0]*( + CM[0,0]*Ia[0]/fi[0] + CM[0,1]*Ia[1]/fi[1])*S[0] + beta[0]*( + CM[0,0]*Is[0]/fi[0] + CM[0,1]*Is[1]/fi[1])*S[0] + gIp*alpha[0]*Ip[0] + gIp*(1-alpha)[0]*Ip[0]
 - gIp*alpha[0]*Ip[0]
 - gIp*(1-alpha)[0]*Ip[0]
 + gIp*alpha[0]*Ip[0] + gIa[0]*Ia[0]

 + gIp*(1-alpha)[0]*Ip[0] + gIs[0]*Is[0]


Confirm: SI2R noise correlation matrix from `inference.pyx`:

    for m in range(M):
        for n in range(M):
            l[m] += beta[m]*CM[m,n]*(Ip[n]+Ia[n]+fsa[n]*Is[n])/fi[n]

    for m in range(M): # only fill in the upper triangular form
        B[0, m, 0, m] = l[m]*s[m]
        B[0, m, 1, m] = -l[m]*s[m]
        B[0, m, 2, m] = 0
        B[0, m, 3, m] = 0
        B[1, m, 1, m] = l[m]*s[m] + gIp[m]*Ip[m]
        B[1, m, 2, m] = - alpha[m]*gIp[m]*Ip[m]
        B[1, m, 3, m] = - balpha[m]*gIp[m]*Ip[m]
        B[2, m, 2, m] = alpha[m]*gIp[m]*Ip[m] + gIa[m]*Ia[m]
        B[2, m, 3, m] = 0
        B[3, m, 3, m] = balpha[m]*gIp[m]*Ip[m] + gIs[m]*Is[m]
